In [7]:
%%time

# Trace evaluation domain and FRI evaluation domain
# G = trace evaluation domain
# L = FRI evaluation domain
# G and L are disjoint
# |G| < |L|
# |G| is a power of 2 (for fast interpolation of polynomial?)
# |L| is a power of 2 (for fast evaluation of polynomial)
# F[P] = Prime field of order P, P is prime
# F[P, *] = Multiplicative subgroup of F[P] = {1, 2, 3, ..., P - 1}
# |F[P, *]| = P - 1
# G and L are subgroups of F[P, *] -> |G| and |L| divides |F[P, *]| = P - 1
from utils import is_prime, is_pow2
from field import find_generator, generate

P = 3 * 2**30 + 1
assert is_prime(P)

### G = trace evaluation domain ###
# T = length of trace
# Find G such that |G| = T
# T must divide P - 1
T = 1024
assert is_pow2(T)

# Generator of F[P, *] - multiplicative group of prime field F[P]
# TODO: g must not be secretly picked by the prover?
g = find_generator(P)
print("Generator of F[P, *] = g =", g)

# Generator of G
# When k divides |F[P, *]|, g^k generates a group of size |F[P, *]| / k
# |F[P, *]| / k = |G| -> k = (P - 1) / |G|
G_gen = pow(g, (P - 1) // T, P)
print("Generator of G =", G_gen)
# TODO: remove?
assert pow(G_gen, T, P) == 1
assert pow(G_gen, T // 2, P) == (-1 % P)

G = generate(G_gen, T, P)

### L = FRI evaluation domain ###
# |L| = EXP_FACTOR * |G|
# L and G are disjoint
EXP_FACTOR = 8
N = EXP_FACTOR * T
assert is_pow2(N)
# |F[P, *]| / k = N -> k = (P - 1) / N
H_gen = pow(g, (P - 1) // N, P)
print("Generator of H =", H_gen)
# TODO: remove?
assert pow(H_gen, N, P) == 1
assert pow(H_gen, N // 2, P) == (-1 % P)

H = generate(H_gen, N, P)

# Coset shifted by g = {g* H}
L = [(g * h) % P for h in H]
assert len(set(L)) == len(H), f'|L| = {len(set(L))}'
assert (set(L) & set(G)) == set(), f'L and G are not disjoint {set(L) & set(G)}'

Generator of F[P, *] = g = 5
Generator of G = 1855261384
Generator of H = 1734477367
CPU times: user 3.16 ms, sys: 1 μs, total: 3.16 ms
Wall time: 3.16 ms


In [9]:
%%time

# FRI - low degree testing on q(x)
import numpy as np
import fri
from field import F
import polynomial
from polynomial import Polynomial
import merkle
import iop
import fft_poly
from utils import is_pow2, is_prime

# TODO: pair f(x) and f(-x) in merkle tree?

# Primitive Nth root of unity
w = H_gen

prover = fri.Prover(
    N = N,
    P = P,
    w = w,
    shift = g,
    exp_factor = EXP_FACTOR,
    eval_domain = L,
)

verifier = fri.Verifier(
    N = N,
    P = P,
    w = w,
    shift = g,
    exp_factor = EXP_FACTOR
)

### Communication channels ###
# Prover to verifier channel
p_chan = iop.Channel(iop.Verifier(verifier))
# Verifier to prover channel
v_chan = iop.Channel(iop.Prover(prover))

### Commit ###
prover.commit(fft_poly.eval(q, H, P, g), p_chan)

print("Merkle root of q = ", prover.merkle_roots[0])
assert prover.merkle_roots[0] == root_q

print("Num codewords =", len(prover.codewords))
print("Last codeword =", prover.codewords[-1])

### Query ###
# Random indexes without replacement
NUM_QUERIES = 50
assert NUM_QUERIES <= N

Q = [int(i) for i in np.random.choice(range(N), size = NUM_QUERIES, replace = False)]
for i in Q:
    verifier.query(i, v_chan)

print("All checks passed ✅️") 
# TODO - attacks
# TODO - probabilities?

Merkle root of q =  4ebd443142249c966cc0f1720b387c6059e327913d199bd59e9ba0a4cb058f3a
Num codewords = 11
Last codeword = [1050761301, 1050761301, 1050761301, 1050761301, 1050761301, 1050761301, 1050761301, 1050761301]
All checks passed ✅️
CPU times: user 832 ms, sys: 0 ns, total: 832 ms
Wall time: 833 ms


In [2]:
%%time

# Trace evaluation domain and FRI evaluation domain
# G = trace evaluation domain
# L = FRI evaluation domain
# G and L are disjoint
# |G| < |L|
# |G| is a power of 2 (to use FFT)
# |L| is a power of 2 (to use FFT)
# F[P] = Prime field of order P, P is prime
# F[P, *] = Multiplicative subgroup of F[P] = {1, 2, 3, ..., P - 1}
# |F[P, *]| = P - 1
# G and L are subgroups of F[P, *] -> |G| and |L| divides |F[P, *]| = P - 1

import numpy as np
from fft import fft, ifft, eval_poly
import fft_poly
from utils import is_prime, is_pow2, min_pow2_gt
from field import F, find_generator, get_primitive_root, generate
import polynomial
from polynomial import Polynomial, X
import stark
import iop
import merkle

### Example claim = trace is all 0 or 1 ###
T = 1024
trace = [int(i) for i in np.random.choice([0, 1], size=T)]

### Setup for evaluation domains - pick prime field and a generator ###
# Prime field F[P]
P = 3 * 2**30 + 1
assert is_prime(P)

# Generator of F[P, *] - multiplicative group of the prime field F[P]
# g must be known by the prover and the verifier
g = find_generator(P)
print("Generator of F[P, *] = g =", g)

### Trace evaluation domain ###
# Let G = trace evaluation domain
# Find G such that |G| = T
# T must be a power of 2 to use FFT
# |G| must divide P - 1 so that |G| is a subgroup of F[P, *]
assert is_pow2(T)
assert (P - 1) % T == 0
trace_eval_domain = generate(get_primitive_root(g, T, P), T, P)

### Trace polynomial ###
# f(x) = trace polynomial
# f(G[i]) = trace[i]
f = fft_poly.interp(trace, trace_eval_domain, P)
assert fft_poly.eval(f, trace_eval_domain, P) == trace, "f(G) != trace"
print("Degree of trace polynomial =", f.degree()) 

### Constraint polynomial ###
# c(x) = constraint polynomial
# trace[i] is either 0 or 1 -> trace[i](trace[i] - 1) = 0 for all G[i]
#                           -> f(x)(f(x) - 1) = 0 for all G[i]
#                           -> Set c(x) = f(x)(f(x) - 1)
c = f*f - f
print("Degree of constraint polynomial =", c.degree())
# Degree of f < T so degree of c < 2 * T
constraint_eval_domain = generate(get_primitive_root(g, 2 * T, P), 2 * T, P)

EXP_FACTOR = 8
N = EXP_FACTOR * T

prover = stark.Prover(
    P = P,
    g = g,
    f = f,
    trace_eval_domain = trace_eval_domain,
    exp_factor = EXP_FACTOR,
    c = c,
    constraint_eval_domain = constraint_eval_domain,
)

x = X(1, lambda x: F(x, P))

verifier = stark.Verifier(
    P = P,
    g = g,
    trace_len = T,
    exp_factor = EXP_FACTOR,
    c = x*x - x,
)

### Communication channels ###
# Prover to verifier channel
p_chan = iop.Channel(iop.Verifier(verifier))
# Verifier to prover channel
v_chan = iop.Channel(iop.Prover(prover))

### STARK and FRI - prover commit ###
prover.commit(p_chan)

### STARK - verifier query ###
NUM_QUERIES = 50
assert NUM_QUERIES <= N

Q = [int(i) for i in np.random.choice(range(N), size = NUM_QUERIES, replace = False)]
for i in Q:
    verifier.query(i, v_chan)

### FRI - verifier query ###
Q = [int(i) for i in np.random.choice(range(N), size = NUM_QUERIES, replace = False)]
for i in Q:
    verifier.fri().query(i, v_chan)

print("All checks passed ✅️") 

Generator of F[P, *] = g = 5
Degree of trace polynomial = 1023
Degree of constraint polynomial = 2046
All checks passed ✅️
CPU times: user 1.73 s, sys: 2.98 ms, total: 1.74 s
Wall time: 1.74 s


In [1]:
from fft import fft_rec, fft, ifft, eval_poly
P = 17
N = 16
g = 3

# Multiplicative group
gs = [pow(g, i, P) for i in range(P - 1)]
assert len(set(gs)) == P - 1

# N-th primitive root
w = pow(g, (P - 1)//N, P)
print("w", w)
# Check w is a N-th primitive root
assert pow(w, N, P) == 1
assert pow(w, N//2, P) == -1 % P

# FFT evaluation domain
ws = [pow(w, i, P) for i in range(N)]
# Example polynomial
f = [i + 1 for i in range(N)]
print("f", f)
print("ws", ws)

e = eval_poly(f, ws, P)
r = fft_rec(f, ws, P)
l = fft(f, ws, P)
i = ifft(l, ws, P)

print("e", e)
print("r", r)
print("l", l)
print("i", i)
assert r == e
assert l == e
assert i == f

# Evaluate a polynomial on a domain larger than the degree of the polynomial
f = [i + 1 for i in range(N//2)] + ([0] * (N//2))
print("f", f)

ys = fft(f, ws, P)
print("ys", ys)

cs = ifft(ys, ws, P)
print("cs", cs)

assert cs == f

w 3
f [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
ws [1, 3, 9, 10, 13, 5, 15, 11, 16, 14, 8, 7, 4, 12, 2, 6]
e [0, 8, 2, 15, 7, 4, 6, 5, 9, 13, 12, 14, 11, 3, 16, 10]
r [0, 8, 2, 15, 7, 4, 6, 5, 9, 13, 12, 14, 11, 3, 16, 10]
l [0, 8, 2, 15, 7, 4, 6, 5, 9, 13, 12, 14, 11, 3, 16, 10]
i [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
f [1, 2, 3, 4, 5, 6, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0]
ys [2, 5, 1, 9, 12, 13, 3, 5, 13, 0, 6, 11, 14, 8, 8, 8]
cs [1, 2, 3, 4, 5, 6, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0]


In [2]:
# N-th root of unity
# Any number w such that w^N = 1 mod P

# Primitive N-th root of unity
# w^N = 1 and w^k != 1 for 0 < k < N 

# Steps to find the N-th primitive root of unity mod P
# 0. Pick a prime P -> This defines the finite field F[P]
#    TODO: Why prime?
#    - nonzero element has a multiplicative inverse
# 1. Consider the multiplicative group F[P]x = {1,2,3,...,P−1} 
# 2. Find a generator g of F[P]x
# 3. Find N = largest power of 2 dividing P−1
#    The subgroup of order N is:
#      H = { g^((P−1)/N)^i  for i = 0 ... N−1 }
# 4. To find a primitive N-th root of unity:
#    Let w = g^((P−1)/N) (g^(P-1) = 1 by Fermat's little theorem)
#    Verify:
#      - w^N = 1
#      TODO: why this check suffices
#      - w^(N/2) = -1

from field import find_generator
from utils import max_log2

# 0. Pick a prime P
# Starkware prime
P = 2**251 + 17 * 2**192 + 1
# P = 17

# 2. Find a generator g
g = find_generator(P)
print("g = ", g)

# 3. Find N = largest power of 2 dividing P - 1
k = max_log2(P - 1)
N = 2**k
print("k =", k)
print("n =", N)

w = pow(g, ((P - 1) // N), P)

assert pow(w, N, P) == 1
assert pow(w, N // 2, P) == (-1 % P)

print(f"{N} th primitive root of unity =", w)

g =  3
k = 192
n = 6277101735386680763835789423207666416102355444464034512896
6277101735386680763835789423207666416102355444464034512896 th primitive root of unity = 145784604816374866144131285430889962727208297722245411306711449302875041684


In [1]:
# A Walk-Through of a Simple zk-STARK Proof

# N = 4
# A = [1, 0, 1, 1]
# g = 4 mod 17
# f(g[i]) = A[i] for i < N
# A[i] = 0 or 1 -> A[i](A[i] - 1) = 0 for i < N -> f(g[i])^2 - f(g[i]) = 0
# c(x) = f(x)^2 - f(x)
# u(x) = ((x-g[0])(x - g[1])...(x - g[N - 1])) = x^N - 1
# p(x) = c(x) / u(x)
# p(x) is polynomial of degree <= deg(c) - N <-> c(x) = 0 for x = g[i] <-> All A[i] = 0 or 1

# Query
# Verifier checks p(x)u(x) = c(x) (query x in L - G, query x in G breaks zero knowledge)
# 1. Verifier queries for z in L - G
# 2. Prover sends p(z) and f(z) and Merkle proofs of p(z) and f(z)
# 3. Verifier checks p(z)u(z) = c(z) and Merkle proofs

# num queries for high probability of fraud detection
# poly degree = d = 5
# evaluation domain L = 100
# num of dishonest evaluation = |L| - d = 95
# probability of getting caught after k queries = 1 - probability of not getting caught after k queries ~ 1 - (d / |L|)^k

# TODO - cheat if p is not a low degree
# TODO - example of false trace?

# Apply FRI to p

Generator of F[P, *] = 5
Generator of G = 1855261384
Generator of L = 1734477367
